In [1]:
import numpy as np
import pandas as pd
from ptrail.core.TrajectoryDF import PTRAILDataFrame
from ptrail.features.kinematic_features import KinematicFeatures
from ptrail.core.Datasets import Datasets
from ptrail.preprocessing.statistics import Statistics

from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, GradientBoostingClassifier
from sklearn.preprocessing import MinMaxScaler
from TestUtils.test_utils import TestUtils
from src.selection.select import Selection
from src.utils.general_utils import Utilities

In [2]:
traffic_dataset = Datasets.load_traffic_data()
ready_dataset = KinematicFeatures.create_distance_column(traffic_dataset)
ready_dataset

------------------------ Dataset Facts ------------------------------

Number of unique Trajectories in the data: 125
Number of points in the data: 44905
Dataset time range: 0 days 00:00:59.900000
Datatype of the DataFrame: <class 'ptrail.core.TrajectoryDF.PTRAILDataFrame'>
Dataset Bounding Box: (34.7107417, 135.4640652, 34.7156517, 135.4702002)

---------------------------------------------------------------------


vehicle_type  velocity  traffic_lane  \
traj_id DateTime                                                        
1371    1900-01-01 07:30:00.000             1      48.0             2   
        1900-01-01 07:30:00.100             1      47.9             2   
        1900-01-01 07:30:00.200             1      47.9             2   
        1900-01-01 07:30:00.300             1      47.9             2   
        1900-01-01 07:30:00.400             1      47.9             2   
...                                       ...       ...           ...   
3357    1900-01-01 07:30:59.500             1      27.7             1   
        1900-01-01 07:30:59.600             1      27.7             1   
        1900-01-01 07:30:59.700             1      29.0             1   
        1900-01-01 07:30:59.800             1      30.3             1   
        1900-01-01 07:30:59.900             1      31.0             1   

                                        lon        lat  kilopost  \
traj_id DateTime                                                   
1371    1900-01-01 07:30:00.000  135.469950  34.710999    3539.5   
        1900-01-01 07:30:00.100  135.469957  34.710991    3532.5   
        1900-01-01 07:30:00.200  135.469963  34.710984    3532.5   
        1900-01-01 07:30:00.300  135.469968  34.710979    3531.5   
        1900-01-01 07:30:00.400  135.469972  34.710974    3530.8   
...                                     ...        ...       ...   
3357    1900-01-01 07:30:59.500  135.468970  34.712177    3697.6   
        1900-01-01 07:30:59.600  135.468975  34.712172    3696.6   
        1900-01-01 07:30:59.700  135.468981  34.712166    3695.6   
        1900-01-01 07:30:59.800  135.468986  34.712160    3694.7   
        1900-01-01 07:30:59.900  135.468992  34.712154    3693.7   

                                 vehicle_length  detected_flag  Distance  
traj_id DateTime                                                          
1371    1900-01-01 07:30:00.000             3.0              0       NaN  
        1900-01-01 07:30:00.100             3.0              0  1.115504  
        1900-01-01 07:30:00.200             3.0              0  0.939478  
        1900-01-01 07:30:00.300             3.0              0  0.763477  
        1900-01-01 07:30:00.400             3.0              0  0.596403  
...                                         ...            ...       ...  
3357    1900-01-01 07:30:59.500             3.5              0  0.737022  
        1900-01-01 07:30:59.600             3.5              1  0.760166  
        1900-01-01 07:30:59.700             3.5              1  0.788944  
        1900-01-01 07:30:59.800             3.5              1  0.835374  
        1900-01-01 07:30:59.900             3.5              1  0.873034  

[44905 rows x 9 columns]

In [3]:
seed_generator = Utilities.generate_pi_seed(1)
seed_vals = [next(seed_generator) for i in range(1)]
shake_percentages = [0.2, 0.4, 0.6]
circle_methods = ['on', 'in']
ml_models = [ExtraTreesClassifier(), GradientBoostingClassifier(), RandomForestClassifier()]
scaler = MinMaxScaler((0, 1))

distance_results = ["on_20%_dist", "on_20%_std", "on_40%_dist", "on_40%_std", "on_60%_dist", "on_60%_std",
                    "in_20%_dist","in_20%_std","in_40%_dist","in_40%_std","in_60%_dist","in_60%_std"]

model_results = ["model", "baseline", "in_20%_f1", "in_40%_f1", "in_60%_f1", "on_20%_f1", "on_40%_f1", "on_60%_f1"]

for seed in seed_vals:
    # Intermediate lists for storing distance and model score values.
    distance_row = []
    model_row = []

    # Set apart 20% data for testing that augmentation process will never see.
    train, test_x, test_y = TestUtils.get_test_train_data(dataset=ready_dataset, seed_val=seed,
                                                          class_col='vehicle_type', k=0.8)
    for model in ml_models:
        base_train_x, base_train_y = TestUtils.get_base_train_x_and_train_y(train, "vehicle_type")
        model_row.append(model.__class__.__name__)
        model_row.append(TestUtils.train_model_and_evaluate(model, scaler.fit_transform(base_train_x), base_train_y,
                                                            scaler.fit_transform(test_x), test_y, seed))
        for shake in shake_percentages:
            for method in circle_methods:
                # Randomly select 30% of trajectories to be augmented.
                selected = Selection.select_randomly(train, seed, k=0.3)

                # Augment the trajectories.
                train_x, train_y = TestUtils.augment_trajectories_using_random_strategy(dataset=train,
                                                                                        percent_to_shake=shake,
                                                                                        ids_to_augment=selected,
                                                                                        circle=method,
                                                                                        n_augmentations=1,
                                                                                        class_col="vehicle_type")

                # Calculate the distances.
                mean, std = TestUtils.find_original_and_augmentation_pairs_and_calculate_differences(train_x, selected)
                f1_score = TestUtils.train_model_and_evaluate(model, scaler.fit_transform(train_x), train_y,
                                                              scaler.fit_transform(test_x), test_y, seed)
                distance_row.append(mean)
                distance_row.append(std)
                model_row.append(f1_score)

    print(distance_row)
    print()
    print(model_row)
    print("----------------------------------------")
    # Append the intermediate lists to the final list.
    distance_results.append(distance_row)
    model_results.append(model_row)


[0.9443616181943876, 0.32965016246073975, 0.928559493270435, 0.320896044065396, 1.0358324988228884, 0.2950693916640335, 1.0353873685693762, 0.2860265013301262, 1.0889518115426413, 0.3254844668835524, 1.064298454006115, 0.2756382043848174, 0.9474367214748377, 0.36430468577879993, 0.9264210845765497, 0.31752134421546535, 1.053533368699006, 0.29314819962160993, 1.039501667733, 0.3035474161033057, 1.0993681831913957, 0.30898772339992625, 1.0722790734752161, 0.29636202335275097, 0.9490198818945649, 0.35199251325132214, 0.9278798344611868, 0.32187702300515786, 1.0359797751856124, 0.29549712264770966, 1.0374041396457379, 0.2979213184886795, 1.0620506856324876, 0.309335846958307, 1.071092432713375, 0.2880309503762299]

['ExtraTreesClassifier', 0.9167, 0.9379, 0.9167, 0.9167, 0.9379, 0.9167, 0.9379, 'GradientBoostingClassifier', 0.5864, 0.7649, 0.7061, 0.7328, 0.7649, 0.7328, 0.7328, 'RandomForestClassifier', 0.9356, 0.8214, 0.8214, 0.8944, 0.7947, 0.7667, 0.7667]
------------------------------